# Test-07
Ejecutado en **Kaggle**.

## Descripción
Se realiza **transfer learning** con diferentes redes derivadas de **EfficientNetB0** como puede ser **EfficientNetB1**, entre otras.

# Configuración general

In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
!pip install cloud_tpu_client

In [4]:
from cloud_tpu_client import Client
c = Client()
c.configure_tpu_version(tf.__version__, restart_type="always")

In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

Running on TPU  ['10.0.0.2:8470']


# Cargando el conjunto de datos

In [6]:
import numpy as np

In [7]:
import os

In [8]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/rn2021q1-itba-cifar100/y_train.npy
/kaggle/input/rn2021q1-itba-cifar100/x_test.npy
/kaggle/input/rn2021q1-itba-cifar100/x_train.npy


In [9]:
x_train_valid = np.load('/kaggle/input/rn2021q1-itba-cifar100/x_train.npy')
y_train_valid = np.load('/kaggle/input/rn2021q1-itba-cifar100/y_train.npy')
x_test = np.load('/kaggle/input/rn2021q1-itba-cifar100/x_test.npy')

# Separando el conjunto de datos

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=15, stratify=y_train_valid)

# Escalando las imágenes

In [12]:
def resize_images(images, height, width):
  """ Resizes images using the bicubic interpolation method and returns them in a
      [0, 255] value mode.
      @param images Group of images
      @param height Height of the output image
      @param width Width of the output image
  """
  resized = tf.image.resize(images / 255, (width, height), method=tf.image.ResizeMethod.BICUBIC) * 255
  resized = tf.cast(resized, tf.int32)
  return resized

In [13]:
x_train_tf = resize_images(x_train, 240, 240)

In [14]:
x_train_tf.shape

TensorShape([40000, 240, 240, 3])

In [15]:
x_valid_tf = resize_images(x_valid, 240, 240)

In [16]:
x_valid_tf.shape

TensorShape([10000, 240, 240, 3])

In [17]:
x_test_tf = resize_images(x_test, 240, 240)

In [18]:
x_test_tf.shape

TensorShape([10000, 240, 240, 3])

# Modelo

In [19]:
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1

In [20]:
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense, Activation, 
                                     BatchNormalization, Reshape, Dropout, InputLayer,
                                     Flatten)

In [21]:
from tensorflow.keras.layers.experimental.preprocessing import (Resizing,
                                                                RandomFlip,
                                                                RandomZoom,
                                                                RandomRotation,
                                                                RandomTranslation)

In [22]:
from tensorflow.keras.models import Sequential

In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [24]:
from tensorflow.keras.optimizers import Adam

In [25]:
from tensorflow import keras

In [34]:
with strategy.scope():
  # Create the model
  model = Sequential()
  model.add(InputLayer(input_shape=(240, 240, 3)))
  model.add(EfficientNetB1(include_top=False, weights='imagenet', input_shape=(240, 240, 3)))
  model.add(GlobalAveragePooling2D())
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(units=100))
  model.add(Activation('softmax'))

  # Compile the model
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=Adam(learning_rate=0.0001),
                metrics=['accuracy'],
                steps_per_execution=30
              )

# Summarize
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb1 (Functional)  (None, 8, 8, 1280)        6575239   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1280)              5120      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               128100    
_________________________________________________________________
activation_1 (Activation)    (None, 100)               0         
Total params: 6,708,459
Trainable params: 6,643,844
Non-trainable params: 64,615
_______________________________________

In [35]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_1.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Create the EarlyStopping callback to stop when not improving during training
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# Create the ReduceLROnPlateau callback to reduce the learning rate when not improving
lr_callback = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=0.001)

# Train the model
epochs = 100
batch_size = 256
model.fit(x_train_tf,
          y_train,
          validation_data=(x_valid_tf, y_valid), 
          callbacks=[mc_callback, es_callback, lr_callback],
          epochs=epochs,
          batch_size=batch_size
          )

Epoch 1/100
157/157 [==============================] - 171s 1s/step - loss: 4.8263 - accuracy: 0.1010 - val_loss: 3.6214 - val_accuracy: 0.2135
Epoch 2/100
157/157 [==============================] - 17s 111ms/step - loss: 1.9566 - accuracy: 0.4917 - val_loss: 1.3745 - val_accuracy: 0.6460
Epoch 3/100
157/157 [==============================] - 17s 111ms/step - loss: 1.2893 - accuracy: 0.6399 - val_loss: 0.7778 - val_accuracy: 0.7710
Epoch 4/100
157/157 [==============================] - 17s 111ms/step - loss: 1.0207 - accuracy: 0.7101 - val_loss: 0.6733 - val_accuracy: 0.7987
Epoch 5/100
157/157 [==============================] - 17s 111ms/step - loss: 0.8218 - accuracy: 0.7610 - val_loss: 0.6198 - val_accuracy: 0.8118
Epoch 6/100
157/157 [==============================] - 17s 111ms/step - loss: 0.7017 - accuracy: 0.7893 - val_loss: 0.5931 - val_accuracy: 0.8226
Epoch 7/100
157/157 [==============================] - 17s 111ms/step - loss: 0.6033 - accuracy: 0.8201 - val_loss: 0.5756 - v

# Evaluación del modelo

In [36]:
import pandas as pd

In [37]:
def generate_submission(predictions, filepath='submission.csv'):
    """ Generate the .csv file to submit in the challenge
        @param predictions Predictions made by the model from the test dataset
        @param filepath Filepath for the file generated
    """
    df = pd.DataFrame(predictions, columns=['label'])
    df.index.name = 'Id'
    df.to_csv(filepath)

In [38]:
# Load the model and show the final metrics
model = keras.models.load_model('model_1.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_tf, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_tf, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.999 Valid: 0.854


In [39]:
y_pred = model.predict(x_test_tf).argmax(axis=-1)

In [40]:
generate_submission(y_pred)